In [2]:
import boto3
import pandas as pd
from io import StringIO
import os, glob
from datetime import datetime, timedelta

In [3]:
arg_date = '2021-03-30'

In [4]:
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

In [5]:
arg_date_dt

datetime.date(2021, 3, 29)

In [6]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('deutsche-boerse-xetra-pds')

In [9]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() == arg_date_dt]

In [10]:
objects

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-03-29/2021-03-29_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [ ]:
 datetime.strptime(obj.key.split("/")[0]

In [7]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]


KeyboardInterrupt



In [ ]:
objects

In [7]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [8]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

Less efficient method (not pythonic) -> more efficient pd.concat([pd.DataFrame([i], columns=['A']) for i in range(5)],
          ignore_index=True)

In [9]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = df_all.append(df, ignore_index=True)

In [10]:
df_all.head(10)

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2021-03-29,07:00,21.180,21.200,21.18,21.20,611,3
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2021-03-29,07:00,48.420,48.420,48.42,48.42,1001,1
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2021-03-29,07:00,25.780,25.900,25.78,25.90,10936,9
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2021-03-29,07:00,198.000,198.000,198.00,198.00,239,1
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2021-03-29,07:00,40.410,40.450,40.40,40.45,6672,11
5,DE000A0JL9W6,VBK,VERBIO VER.BIOENERGIE ON,Common stock,EUR,2504343,2021-03-29,07:00,35.800,35.800,35.80,35.80,2965,2
6,DE000A0LD2U1,AOX,ALSTRIA OFFICE REIT-AG,Common stock,EUR,2504379,2021-03-29,07:00,14.040,14.040,14.04,14.04,626,1
7,DE000A0LD6E6,GXI,GERRESHEIMER AG,Common stock,EUR,2504380,2021-03-29,07:00,83.650,83.650,83.35,83.50,3251,24
8,DE000A0S8488,HHFA,HAMBURG.HAFEN LOG.A-SP NA,Common stock,EUR,2504409,2021-03-29,07:00,19.640,19.640,19.64,19.64,1354,1
9,DE000A0WMPJ6,AIXA,AIXTRON SE NA O.N.,Common stock,EUR,2504428,2021-03-29,07:00,18.885,18.885,18.86,18.88,10523,34


In [11]:
columns = ["ISIN", "Mnemonic", "Date", "Time", "StartPrice", "EndPrice", "MinPrice", "MaxPrice", "TradedVolume"]
df_all = df_all.loc[:, columns]

In [12]:
df_all

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume
0,AT0000A0E9W5,SANT,2021-03-29,07:00,21.180,21.200,21.180,21.200,611
1,DE000A0DJ6J9,S92,2021-03-29,07:00,48.420,48.420,48.420,48.420,1001
2,DE000A0D6554,NDX1,2021-03-29,07:00,25.780,25.900,25.780,25.900,10936
3,DE000A0D9PT0,MTX,2021-03-29,07:00,198.000,198.000,198.000,198.000,239
4,DE000A0HN5C6,DWNI,2021-03-29,07:00,40.410,40.450,40.400,40.450,6672
...,...,...,...,...,...,...,...,...,...
383434,DK0010268606,VWS,2021-04-01,15:40,176.340,176.340,176.340,176.340,50
383435,DE000TUAG000,TUI1,2021-04-01,15:42,4.483,4.483,4.483,4.483,2039
383436,DE0006599905,MRK,2021-04-01,15:42,145.550,145.550,145.550,145.550,101
383437,DK0010268606,VWS,2021-04-01,15:43,176.340,176.340,176.340,176.340,50


In [13]:
df_all.dropna(inplace=True)

In [14]:
df_all.shape

(383439, 9)

## Get opening price per ISIN and day

In [15]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [16]:
df_all[df_all['ISIN']=='DE0007568578']

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume,opening_price
957,DE0007568578,F3C,2021-03-29,07:04,24.60,24.65,24.60,24.65,4399,24.60
2899,DE0007568578,F3C,2021-03-29,07:05,24.75,24.80,24.75,24.85,1412,24.60
3173,DE0007568578,F3C,2021-03-29,07:06,24.80,24.80,24.80,24.80,161,24.60
3459,DE0007568578,F3C,2021-03-29,07:07,24.70,24.85,24.60,24.85,1581,24.60
4155,DE0007568578,F3C,2021-03-29,07:10,24.80,24.90,24.80,24.90,250,24.60
...,...,...,...,...,...,...,...,...,...,...
379346,DE0007568578,F3C,2021-04-01,15:27,26.90,26.90,26.90,26.90,57,27.25
379670,DE0007568578,F3C,2021-04-01,15:28,26.75,26.75,26.75,26.75,250,27.25
380034,DE0007568578,F3C,2021-04-01,15:29,26.75,26.70,26.70,26.85,278,27.25
380340,DE0007568578,F3C,2021-04-01,15:30,26.60,26.60,26.60,26.60,145,27.25


## Get closing price per ISIN and day

In [17]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [18]:
df_all[df_all['ISIN']=='DE0007568578']

,ISIN,Mnemonic,Date,Time,StartPrice,EndPrice,MinPrice,MaxPrice,TradedVolume,opening_price,closing_price
957,DE0007568578,F3C,2021-03-29,07:04,24.60,24.65,24.60,24.65,4399,24.60,24.50
2899,DE0007568578,F3C,2021-03-29,07:05,24.75,24.80,24.75,24.85,1412,24.60,24.50
3173,DE0007568578,F3C,2021-03-29,07:06,24.80,24.80,24.80,24.80,161,24.60,24.50
3459,DE0007568578,F3C,2021-03-29,07:07,24.70,24.85,24.60,24.85,1581,24.60,24.50
4155,DE0007568578,F3C,2021-03-29,07:10,24.80,24.90,24.80,24.90,250,24.60,24.50
...,...,...,...,...,...,...,...,...,...,...,...
379346,DE0007568578,F3C,2021-04-01,15:27,26.90,26.90,26.90,26.90,57,27.25,26.75
379670,DE0007568578,F3C,2021-04-01,15:28,26.75,26.75,26.75,26.75,250,27.25,26.75
380034,DE0007568578,F3C,2021-04-01,15:29,26.75,26.70,26.70,26.85,278,27.25,26.75
380340,DE0007568578,F3C,2021-04-01,15:30,26.60,26.60,26.60,26.60,145,27.25,26.75


## Aggregations

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [20]:
df_all

opening_price_eur  closing_price_eur  \
ISIN         Date                                               
AT00000FACC2 2021-03-29              9.040              8.990   
             2021-03-30              9.040              9.120   
             2021-03-31              9.100              9.100   
             2021-04-01              9.180              9.370   
AT0000606306 2021-03-29             18.500             18.400   
...                                    ...                ...   
XS2265370234 2021-04-01             22.378             22.368   
XS2284324667 2021-03-29             24.726             24.932   
             2021-03-30             25.054             24.916   
             2021-03-31             25.034             24.970   
             2021-04-01             24.804             24.676   

                         minimum_price_eur  maximum_price_eur  \
ISIN         Date                                               
AT00000FACC2 2021-03-29              8.980              9.040   
             2021-03-30              9.000              9.120   
             2021-03-31              9.100              9.100   
             2021-04-01              9.180              9.430   
AT0000606306 2021-03-29             18.290             18.540   
...                                    ...                ...   
XS2265370234 2021-04-01             22.368             22.378   
XS2284324667 2021-03-29             24.726             25.180   
             2021-03-30             24.864             25.054   
             2021-03-31             24.770             25.056   
             2021-04-01             24.494             24.848   

                         daily_traded_volume  
ISIN         Date                             
AT00000FACC2 2021-03-29                  497  
             2021-03-30                  873  
             2021-03-31                    0  
             2021-04-01                  651  
AT0000606306 2021-03-29                 3200  
...                                      ...  
XS2265370234 2021-04-01                    0  
XS2284324667 2021-03-29                 7596  
             2021-03-30                 2598  
             2021-03-31                 4537  
             2021-04-01                 4594  

[11985 rows x 5 columns]

percent change prev closing

In [21]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [22]:
df_all

opening_price_eur  closing_price_eur  \
ISIN         Date                                               
AT00000FACC2 2021-03-29              9.040              8.990   
             2021-03-30              9.040              9.120   
             2021-03-31              9.100              9.100   
             2021-04-01              9.180              9.370   
AT0000606306 2021-03-29             18.500             18.400   
...                                    ...                ...   
XS2265370234 2021-04-01             22.378             22.368   
XS2284324667 2021-03-29             24.726             24.932   
             2021-03-30             25.054             24.916   
             2021-03-31             25.034             24.970   
             2021-04-01             24.804             24.676   

                         minimum_price_eur  maximum_price_eur  \
ISIN         Date                                               
AT00000FACC2 2021-03-29              8.980              9.040   
             2021-03-30              9.000              9.120   
             2021-03-31              9.100              9.100   
             2021-04-01              9.180              9.430   
AT0000606306 2021-03-29             18.290             18.540   
...                                    ...                ...   
XS2265370234 2021-04-01             22.368             22.378   
XS2284324667 2021-03-29             24.726             25.180   
             2021-03-30             24.864             25.054   
             2021-03-31             24.770             25.056   
             2021-04-01             24.494             24.848   

                         daily_traded_volume  prev_closing_price  
ISIN         Date                                                 
AT00000FACC2 2021-03-29                  497                 NaN  
             2021-03-30                  873               8.990  
             2021-03-31                    0               9.120  
             2021-04-01                  651               9.100  
AT0000606306 2021-03-29                 3200                 NaN  
...                                      ...                 ...  
XS2265370234 2021-04-01                    0              22.474  
XS2284324667 2021-03-29                 7596                 NaN  
             2021-03-30                 2598              24.932  
             2021-03-31                 4537              24.916  
             2021-04-01                 4594              24.970  

[11985 rows x 6 columns]

In [23]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [24]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [25]:
df_all = df_all.round(decimals=2)

In [27]:
df_all.reset_index(inplace=True)

In [28]:
df_all = df_all[df_all.Date >= arg_date]

In [29]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
1,AT00000FACC2,2021-03-30,9.04,9.12,9.00,9.12,873,1.45
2,AT00000FACC2,2021-03-31,9.10,9.10,9.10,9.10,0,-0.22
3,AT00000FACC2,2021-04-01,9.18,9.37,9.18,9.43,651,2.97
5,AT0000606306,2021-03-30,18.74,18.81,18.74,18.92,2900,2.23
6,AT0000606306,2021-03-31,18.90,18.73,18.73,18.98,3240,-0.43
...,...,...,...,...,...,...,...,...
11979,XS2265370234,2021-03-31,22.47,22.47,22.47,22.47,67,NaN
11980,XS2265370234,2021-04-01,22.38,22.37,22.37,22.38,0,-0.47
11982,XS2284324667,2021-03-30,25.05,24.92,24.86,25.05,2598,-0.06
11983,XS2284324667,2021-03-31,25.03,24.97,24.77,25.06,4537,0.22
